<h1 align="center">Assignment 4</h1>
<h2 align="center">Programming Machine Learning Applications [DSC 478]</h2>
<h3 align="center">Jonesh Shrestha (2222011)</h3>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itemBasedRec import *

### 1. **Item-Based Joke Recommendation**
#### For this problem you will use a modified version of the item-based recommender algorithm from Ch. 14 of Machine Learning in Action and use it on joke ratings data based on Jester Online Joke Recommender System. The modified version of the code is provided in the module itemBasedRec.py. Most of the module will be used as is, but you will add some additional functionality. The data set contains two files. The file "**modified_jester_data.csv**" contains the ratings on 100 jokes by 1000 users (each row is a user profile). The ratings have been normalized to be between 1 and 21 (a 20-point scale), with 1 being the lowest rating. A zero indicated a missing rating. The file "**jokes.csv**" contains the joke ids mapped to the actual text of the jokes. Your tasks in this problem are the following. 

#### a. **[10 pts]** Load in the joke ratings data and the joke text data into appropriate data structures. Use the "**recommend**" function to provide top 5 joke recommendations for **users with id 117** using both **Pearson and cosine similarity** measures. Note the differences. Use the standard item-based collaborative filtering (based on the rating prediction function "**standEst**"). Next, find the top 5 recommendations for **user with id 441** only with Pearson similarity using both the standard estimator and the SVD-based version (using "**svdEst**" as the prediction engine) to generate these recommendations. Note the differences. When outputting recommendations, you should show both the id and the text of the recommended jokes (in decreasing order of predicted rating) as well as the predicted ratings for each.

In [3]:
# jokes_df = pd.read_csv('jokes/jokes.csv', header=None, index_col=0)
# jokes_df.head(5)

jokes = load_jokes('jokes/jokes.csv')
print(jokes[:5])

['A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer\'s disease". The man replies "Well thank God I don\'t have cancer!"'
 'This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That\'s an awfully big word for a ten year old."'
 "Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert."
 "Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it."
 "Q. What's O. J. Simpson's Internet address? A.\tSlash slash backslash slash slash escape."]


In [4]:
jester_data_df = pd.read_csv('jokes/modified_jester_data.csv', header=None)
jester_data_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,3.18,19.79,1.34,2.84,3.48,2.50,1.15,15.17,2.02,6.24,...,13.82,0.00,0.00,0.00,0.00,0.00,5.37,0.00,0.00,0.00
1,15.08,10.71,17.36,15.37,8.62,1.34,10.27,5.66,19.88,20.22,...,13.82,6.05,10.71,18.86,10.81,8.86,14.06,11.34,6.68,12.07
2,0.00,0.00,0.00,0.00,20.03,20.27,20.03,20.27,0.00,0.00,...,0.00,0.00,0.00,20.08,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,19.35,0.00,0.00,12.80,19.16,8.18,17.21,0.00,12.84,...,0.00,0.00,0.00,11.53,0.00,0.00,0.00,0.00,0.00,0.00
4,19.50,15.61,6.83,5.61,12.36,12.60,18.04,15.61,10.56,16.73,...,16.19,16.58,15.27,16.19,16.73,12.55,14.11,17.55,12.80,12.60


In [5]:
jester_data_np = jester_data_df.to_numpy()
jester_data_np

array([[ 3.18, 19.79,  1.34, ...,  0.  ,  0.  ,  0.  ],
       [15.08, 10.71, 17.36, ..., 11.34,  6.68, 12.07],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [16.58, 16.63, 15.85, ...,  0.  ,  0.  ,  0.  ],
       [ 3.67,  4.45,  3.67, ...,  3.77,  3.77,  3.28],
       [ 9.88, 11.73,  9.16, ...,  0.  ,  0.  ,  0.  ]], shape=(1000, 100))

In [6]:
def recommendation_output(recommend, jokes=jokes):
    for id, score in recommend:
        print(f'Joke ID: {id}')
        print(f'Score: {score}')
        print(f'Joke Text: {get_joke_text(jokes, id)}\n')

In [7]:
pearson_recommend = recommend(dataMat=jester_data_np, user=117, N=5, simMeas=pearsonSim, estMethod=standEst)
recommendation_output(pearson_recommend)

Joke ID: 97
Score: 10.611085062642834
Joke Text: Age and Womanhood1. Between the ages of 13 and 18 ... She is like Africa virgin and unexplored. 2. Between the ages of 19 and 35 ... She is like Asia hot and exotic. 3. Between the ages of 36 and 45 ... She is like America fully explored breathtakingly beautiful and free with her resources.4. Between the ages of 46 and 56 ...She is like Europe exhausted but still has points of interest. 5. After 56 she is like Australia ...Everybody knows it's down there but who gives a damn?

Joke ID: 99
Score: 10.60795594948322
Joke Text: Q: What's the difference between greeting a Queen and greeting thePresident of the United  States?A: You only have to get on one knee to greet the queen.

Joke ID: 92
Score: 10.59028160354093
Joke Text: Reaching the end of a job interview the human resources person asked a young engineer fresh out of Stanford"And what starting salary were you looking for?"The engineer said "In the neighborhood of $125000 a year depend

In [8]:
cosine_recommend = recommend(dataMat=jester_data_np, user=117, N=5, simMeas=cosineSim, estMethod=standEst)
recommendation_output(cosine_recommend)

Joke ID: 97
Score: 10.568879679136282
Joke Text: Age and Womanhood1. Between the ages of 13 and 18 ... She is like Africa virgin and unexplored. 2. Between the ages of 19 and 35 ... She is like Asia hot and exotic. 3. Between the ages of 36 and 45 ... She is like America fully explored breathtakingly beautiful and free with her resources.4. Between the ages of 46 and 56 ...She is like Europe exhausted but still has points of interest. 5. After 56 she is like Australia ...Everybody knows it's down there but who gives a damn?

Joke ID: 99
Score: 10.566963805972222
Joke Text: Q: What's the difference between greeting a Queen and greeting thePresident of the United  States?A: You only have to get on one knee to greet the queen.

Joke ID: 92
Score: 10.56570515577353
Joke Text: Reaching the end of a job interview the human resources person asked a young engineer fresh out of Stanford"And what starting salary were you looking for?"The engineer said "In the neighborhood of $125000 a year depen

In [9]:
# for user with id 441 only with Pearson similarity using both the standard estimator
standEst_pearson_recommend = recommend(dataMat=jester_data_np, user=441, N=5, simMeas=pearsonSim, estMethod=standEst)
recommendation_output(standEst_pearson_recommend)

Joke ID: 79
Score: 13.971888175175714
Joke Text: Hillary Bill Clinton and the Pope are sitting together on an airplane. Bill says "I could throw one thousand dollar bill out of this plane and make one person very happy."Hillary says "I could throw 10 hundred dollar bills out of the plane and make 10 people very happy."The Pope chips in and says "I could throw Bill out of the airplane and make the whole country happy."

Joke ID: 5
Score: 13.965840113937233
Joke Text: Bill & Hillary are on a trip back to Arkansas. They're almost out of gas so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. As Bill pulls the car onto the road he turns to Hillary and says 'Now aren't you glad you married me and not him ? You could've been the wife of a grease monkey !' To which Hi

In [10]:
# the SVD-based version
svdEst_pearson_recommend = recommend(dataMat=jester_data_np, user=441, N=5, simMeas=pearsonSim, estMethod=svdEst)
recommendation_output(svdEst_pearson_recommend)

Joke ID: 45
Score: 14.299797597297259
Joke Text: A couple has been married for 75 years. For the husband's 95thbirthday his wife decides to surprise him by hiring a prostitute.That day the doorbell rings. The husband uses his walker to get to the door and opens it. A 21-year-old in a latex outfit smiles and says "Hi I here to give you super sex!" The old man says "I'll take the soup."

Joke ID: 5
Score: 14.225833193529487
Joke Text: Bill & Hillary are on a trip back to Arkansas. They're almost out of gas so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. As Bill pulls the car onto the road he turns to Hillary and says 'Now aren't you glad you married me and not him ? You could've been the wife of a grease monkey !' To which Hillary replied 'No Bill. If I would

_First I imported the necessary libraries along with all the functions from the module itemBasedRec.py. I used load_jokes function to load the jokes dataset. Then used pandas to load the modified_jester_data as dataframe and converted to numpy array as recommend() function takes dataMat as numpy array. I also created a recommendation_output that uses jokes and the output from recommend() function to print joke id, score and joke text as required. for user with id 117, the recommend function with prediction fuction standEst with both cosine and pearson similarity gave similar top recommendations with jokeid 97, 99, 92, 75 appearing on both the lists. However pearson recommended 80 while cosine recommended 88. This is due to Pearson adjusts for each user's rating behaviour and bias, whereas cosine only measures the similarity between rating patterns without accounting for user's bias._

_Again for the user with id 441, the recommend function using the standard estimator (standEst) and the SVD-based estimator (svdEst), both with Pearson similarity, returned different sets of jokes, and only the joke with id 5 appeared in both lists. This happens because the standard estimator only relies on direct rating overlap between items, which is limited in a sparse rating matrix where most ratings are 0. On the other hand, the SVD-based estimator reduces the dimensionality of the user-item matrix and represents jokes in a latent feature space, which allows it to infer preferences even when the overlap in ratings is weak or missing (i.e., when many values are 0). As a result, the SVD-based recommendations differ._

#### b. **[10 pts]** Complete the definition for the function "**test**" (the stub is provided in the **itemBasedRec** module). This function iterates over all users and for each performs evaluation (by calling the provided "**cross_validate_user**" function) and returns the error information necessary to compute Mean Absolute Error (MAE). Use this function to perform evaluation (with 20% test-ratio for each user) comparing MAE results using the rating prediction function "standEst" with results using the "svdEst" prediction function (in both cases using Pearson similarity measure). Note that this may take several minutes depending on your computational environment. [**Note**: See comments provided in the module for hints on accomplishing these tasks.]

In [13]:
%%time
MAE_standEst = test(dataMat=jester_data_df, test_ratio=0.2, estMethod=standEst, simMeas=pearsonSim)
print(f'Mean Absolute Error for standEst using Pearson Similarity: {MAE_standEst:.4f}')

Mean Absolute Error for standEst using Pearson Similarity: 3.6934
CPU times: user 23.7 s, sys: 61.1 ms, total: 23.7 s
Wall time: 24.3 s


In [16]:
%%time
MAE_svdEst = test(dataMat=jester_data_df, test_ratio=0.2, estMethod=svdEst, simMeas=pearsonSim)
print(f'Mean Absolute Error for svdEst using Pearson Similarity: {MAE_svdEst:.4f}')

Mean Absolute Error with svdEst: 3.6643


_First I completed the test() function by looping through each user in dataMat which is the length of dataMat(row), the called the cross_validate_user() function to get the user_error and user_count which I sum across all the users to get the MAE on the test data by dividing total_error by total_count. Then I used the 20% of the jester_data_np and calculated for standEst and svdEst estimators on pearsonSim. This gives MAE of 3.6969 for standEst and 3.6643 for svdEst_

#### c. **[15 pts]** Write a new function **"print_most_similar_jokes"** which outputs the most similar jokes (based on user ratings) to a specified query joke. Your function should take as input the joke ratings data, a query joke id, a parameter **k** for the number similar jokes, and a similarity metric function. It should output the text of the query joke as well as the texts of the top k most similar jokes in decreasing order of similarity (you should also provide the similarity values). Test your function as follows:

     * Show the top 3 most similar jokes to joke with id 9 using Pearson similarity.
     * Show the top 3 most similar jokes to Joke with id 9 using cosine similarity.

[**Note**: see comments at the end of the **itemBasedRec** module as well as comments for the provided stub function.]

In [11]:
print_most_similar_jokes(jester_data_df, jokes, 9, 3, pearsonSim)

Selected joke: Joke # 9
Two cannibals are eating a clown one turns to other and says: "Does this taste funny to you? 

Top 3 recommendations are : 

Joke # 54 (Similarity: 0.6930598395934301):
A woman has twins and gives them up for adoption.  One of them goes to a family in Egypt and is named "Amal."  The other goes to a  family in Spain; they name him "Juan."  Years later Juan sends a picture of himself to his mom.  Upon receiving the picture she tells her husband that she wishes she also had a picture of Amal.  Her husband responds "But they are twins-if you've seen Juan you've seen   Amal.

Joke # 43 (Similarity: 0.6925114947798079):
A horse walks into a bar. Bartender says:"So why the long face?"

Joke # 37 (Similarity: 0.6881434530285019):
"May I take your order?" the waiter asked. "Yes how do you prepare your chickens?" "Nothing special sir" he replied. "We just tell them straight out that they're going to die."



In [12]:
print_most_similar_jokes(jester_data_np, jokes, 9, 3, cosineSim)

Selected joke: Joke # 9
Two cannibals are eating a clown one turns to other and says: "Does this taste funny to you? 

Top 3 recommendations are : 

Joke # 49 (Similarity: 0.9567814498620507):
A guy goes into confession and says to the priest "Father I'm 80 years old widower with 11 grandchildren. Last night I met two beautiful flight attendants. They took me home and I made love to both of them. Twice."The priest said: "Well my son when was the last time you were in confession?" "Never Father I'm Jewish." "So then why are you telling me?" "I'm telling everybody."

Joke # 37 (Similarity: 0.955529834986665):
"May I take your order?" the waiter asked. "Yes how do you prepare your chickens?" "Nothing special sir" he replied. "We just tell them straight out that they're going to die."

Joke # 87 (Similarity: 0.9554097405075856):
A Czechoslovakian man felt his eyesight was growing steadily worse and felt it was time to go see an optometrist. The doctor started with some simple testing and s

#### d. **[15 pts]** The implementation of item-based collaborative filtering provided in the module is not scalable since for each prediction it attempts to compute pairwise similarities among all items. Develop your own item-based collaborative filtering recommender that uses a model-based approach (separating the training and the prediction tasks). In the training component, item-item similarities for all pairs of items are computed and stored in an appropriate data structure such as a pairwise similarity matrix. Your training function should be able to use different similarity functions (passed as a parameter) including cosine Similarity or Pearson correlation. The prediction (or estimation) function should take as parameters a target user, an item, a value of **k**, and the similarities matrix computed in the training phase. It should then return the predicted rating on the target item for the target user. The predicted rating should be the weighted average of the target user's ratings on the **k** most similar items to the target item (obtained from the similarity matrix). Demonstrate that your function works by computing predicted ratings for **users 117 and 441**, using **k = 10**, on top two items recommended to each user on part a (using both Pearson and cosine similarities).

In [26]:
def similarity_matrix(dataMat, metric):
    dataMat = np.array(dataMat)
    n = np.shape(dataMat)[1]
    # create a matrix of shape n x n 
    simMat = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            overLap = np.nonzero(np.logical_and(dataMat[:,i]>0, dataMat[:,j]>0))[0]
            if len(overLap) == 0: 
                similarity = 0
            else: 
                similarity = metric(dataMat[overLap,i], dataMat[overLap,j])
            simMat[i,j] = similarity
            simMat[j,i] = similarity
    return simMat

In [27]:
np.set_printoptions(precision=3, edgeitems=5)
simMat = similarity_matrix(jester_data_df, metric = pearsonSim)
print(simMat)

[[1.    0.655 0.671 0.636 0.561 ... 0.577 0.585 0.555 0.612 0.542]
 [0.655 1.    0.604 0.642 0.563 ... 0.552 0.569 0.604 0.566 0.615]
 [0.671 0.604 1.    0.685 0.62  ... 0.582 0.557 0.555 0.531 0.603]
 [0.636 0.642 0.685 1.    0.641 ... 0.639 0.588 0.65  0.573 0.677]
 [0.561 0.563 0.62  0.641 1.    ... 0.545 0.531 0.594 0.562 0.609]
 ...
 [0.577 0.552 0.582 0.639 0.545 ... 1.    0.706 0.657 0.691 0.618]
 [0.585 0.569 0.557 0.588 0.531 ... 0.706 1.    0.631 0.673 0.556]
 [0.555 0.604 0.555 0.65  0.594 ... 0.657 0.631 1.    0.634 0.711]
 [0.612 0.566 0.531 0.573 0.562 ... 0.691 0.673 0.634 1.    0.58 ]
 [0.542 0.615 0.603 0.677 0.609 ... 0.618 0.556 0.711 0.58  1.   ]]


In [ ]:
def item_based_predict(dataMat, simMat, user, item, k):
    # get all items (jokes) the user has rated 
    user_ratings = dataMat[user, :]
    # indices of the rated items
    rated_items_idx = np.where(user_ratings > 0)[0]

    # if user has no ratings return 0
    if len(rated_items_idx) == 0:
        return 0

    # get how similar the item is to all other items (rated items)
    item_sims = simMat[item, rated_items_idx]

    # sort and get the top k similar items index in reverse
    top_k_idx = np.argsort(item_sims)[::-1][k]
    # get the similarity value from index
    top_k_sims = items_sims[top_k_idx]
    # get the user ratings for these items
    top_k_ratings = user_ratings[rated_items_idx[top_k_idx]]

    # if sum of the top k similarities is 0 return 0
    if np.sum(top_k_sims) == 0:
        return 0
    # calculate the weighted sum
    pred = np.dot(top_k_sims, top_k_ratings) / np.sum(top_k_sims)
    return user, pred